# Problem 5

In [1]:
%pylab inline

import numpy as np
import pandas as pd
import itertools as it

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

Populating the interactive namespace from numpy and matplotlib


In [2]:
columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "acceptability"]
data = pd.read_csv("car-evaluation.arff", names=columns)
data = data.drop(['doors'], axis = 1)
data.head()

,buying,maint,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,small,low,3
1,vhigh,vhigh,2,small,med,3
2,vhigh,vhigh,2,small,high,3
3,vhigh,vhigh,2,med,low,3
4,vhigh,vhigh,2,med,med,3


In [3]:
data["safety"].replace({"low": 0, "med": 1, "high": 2}, inplace=True)
data["lug_boot"].replace({"small": 0, "med": 1, "big": 2}, inplace=True)
data["persons"].replace({"2": 0, "4": 1, "more": 2}, inplace=True)
data["maint"].replace({"low": 0, "med": 1, "high": 2, "vhigh": 3}, inplace=True)
data["buying"].replace({"low": 0, "med": 1, "high": 2, "vhigh": 3}, inplace=True)
data.head()

,buying,maint,persons,lug_boot,safety,acceptability
0,3,3,0,0,0,3
1,3,3,0,0,1,3
2,3,3,0,0,2,3
3,3,3,0,1,0,3
4,3,3,0,1,1,3


In [4]:
classifiers = [tree.DecisionTreeClassifier, RandomForestClassifier, ExtraTreesClassifier]
params = [
    {
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, None],
    },
    {
        "n_estimators": [10, 50],
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, None],
    },
    {
        "n_estimators": [10, 50],
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, None],
    },
]

for classifier_class, param in zip(classifiers, params):
    best = 0, {}
    for param_combination in [dict(zip(param, prod)) for prod in it.product(*(param[varName] for varName in param))]:
        classifier = classifier_class(**param_combination)
        repetitions = 100
        test_scores = 0
        for i in range(repetitions):
            data['train'] = np.random.uniform(0, 1, len(data))

            data_train = data[data['train'] <= 0.7]
            data_test = data[data['train'] > 0.7]

            y = data_train['acceptability']
            X = data_train.drop('acceptability', axis = 1)

            classifier = classifier.fit(X, y)

            y = data_test['acceptability']
            X = data_test.drop('acceptability', axis = 1)
            test_scores += classifier.score(X, y)
        
        avg_score = 100.0*test_scores/repetitions
        if avg_score > best[0]: best = avg_score, param_combination

    print "Best average score using {}:\n\tparams: {}\n\taverage score: {:.2f}%".format(classifier_class, best[1], best[0])

Best average score using <class 'sklearn.tree.tree.DecisionTreeClassifier'>:
	params: {'criterion': 'entropy', 'max_depth': 10}
	average score: 93.32%
Best average score using <class 'sklearn.ensemble.forest.RandomForestClassifier'>:
	params: {'n_estimators': 50, 'criterion': 'entropy', 'max_depth': 10}
	average score: 93.97%
Best average score using <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>:
	params: {'n_estimators': 50, 'criterion': 'gini', 'max_depth': 10}
	average score: 93.61%
